In [55]:
# Main
import numpy as np
import pandas as pd
import datetime as dt
from statsmodels.tsa.tsatools import lagmat

# Modeling
import statsmodels.api as sm
from statsmodels.tsa.tsatools import add_trend
from statsmodels.regression.linear_model import OLS

# Reporting
from statsmodels.iolib.summary2 import summary_col
from IPython.display import display, HTML

# Plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [6]:
# renormalize REERs at 2000q1 = 100
data = pd.read_csv('data/dataset.csv', index_col=0, parse_dates=True)
data = 100*data/data.iloc[0,:]

# Changing data
dataLogs = data.apply(np.log, axis=1)
dataPtC = dataLogs.diff()[1:]

In [7]:
# This replicates the results above with the ADF class
lags = 1 # dynamically scalable
diff = dataLogs['Armenia'].diff(1)
d_y = diff[1+lags:]
y_1 = dataLogs['Armenia'].shift(1).rename('Level.L1')
d_y_lags = pd.DataFrame(lagmat(diff, maxlag=lags), index=diff.index,
                        columns=['Diff.L'+str(i) for i in range(1, lags+1)])
X = sm.add_constant(pd.concat([y_1, d_y_lags], axis=1))[1+lags:]
res = OLS(d_y, exog=X).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Armenia   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     4.988
Date:                Thu, 22 Jul 2021   Prob (F-statistic):            0.00752
Time:                        02:51:28   Log-Likelihood:                 596.91
No. Observations:                 249   AIC:                            -1188.
Df Residuals:                     246   BIC:                            -1177.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0720      0.043      1.684      0.093      -0.012       0.156
Level.L1      -0.0147      0.009     -1.664      0.097      -0.032       0.003
Diff.L1        0.1726      0.063      2.753      0.006       0.049       0.296
==============================================================================
Omnibus:                      128.453   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2270.192
Skew:                          -1.583   Prob(JB):                         0.00
Kurtosis:                      17.450   Cond. No.                         221.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Model overview:

$\Delta y_t= \alpha_0 + \sum_{j=1}^{q} \alpha_j \Delta y_{t-j} + \beta_1 y_{t-1} + (\beta_2-\beta_1)[t\geq t^*] y_{t-1} + \epsilon_t$

When: lags = 1; EAEU = 1*$[t\geq t^*]$; then:

$\Delta y_t= \alpha_0 + \alpha_1 \Delta y_{t-1} + \beta_1 y_{t-1} + (\beta_2-\beta_1)\small{\textrm{EAEU}}y_{t-1} + \epsilon_t$

In [39]:
def mainModel(endog, t_star, lags=1):
    diff = endog.diff(1)
    d_y = diff[1+lags:]
    EAEU = (endog.index > t_star).astype(int)
    y_1 = endog.shift(1).rename('β₁')
    y_1_post = pd.DataFrame(y_1*EAEU).add_prefix('β₂-')
    d_y_lags = pd.DataFrame(lagmat(diff, maxlag=lags), index=diff.index,
                            columns=[f'α{str(chr(8320+i))}' for i in range(1, lags+1)])
    X = sm.add_constant(pd.concat([y_1, y_1_post, d_y_lags], axis=1))[1+lags:]
    res = OLS(d_y, exog=X).fit()
    return res

In [40]:
# ADF test with post-EAEU dummy
log_y = dataLogs['Armenia']
t_star = '2015-01-01'
lags = 1

results = mainModel(log_y, t_star, lags)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Armenia   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     3.314
Date:                Thu, 22 Jul 2021   Prob (F-statistic):             0.0207
Time:                        03:46:04   Log-Likelihood:                 596.92
No. Observations:                 249   AIC:                            -1186.
Df Residuals:                     245   BIC:                            -1172.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0702      0.048      1.471      0.143      -0.024       0.164
β₁            -0.0143      0.010     -1.438      0.152      -0.034       0.005
β₂-β₁      -5.967e-05      0.001     -0.084      0.933      -0.001       0.001
α₁             0.1722      0.063      2.734      0.007       0.048       0.296
==============================================================================
Omnibus:                      129.114   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2289.170
Skew:                          -1.594   Prob(JB):                         0.00
Kurtosis:                      17.508   Cond. No.                         233.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
# Running main model for all countries
t_star = '2015-01-01'
lags = 1
modelResults = []

for country in dataLogs:
    modelResults.append(mainModel(dataLogs[country], t_star, lags))

In [62]:
# info = {
#     "N":lambda x:"{0:d}".format(int(x.nobs)),
#     'Pseudo_R2': lambda x:"{:.2f}".format(x.prsquared)
# }
models = modelResults
modelNames = list(dataLogs.columns)

view = summary_col(models, model_names=modelNames, 
                   stars=True, float_format='%.3f')
view.add_title('Main Model Results')
view.extra_txt = ['* p<.1, ** p<.05, ***p<.01']
view

,Armenia,Belarus,Kazakhstan,Kyrgyz Republic,Moldova,Russian Federation,Tajikistan,Ukraine
const,0.070,0.205**,0.222***,0.088,0.067*,0.158***,0.262***,0.349***
,(0.048),(0.080),(0.063),(0.060),(0.035),(0.049),(0.072),(0.117)
β₁,-0.014,-0.047**,-0.046***,-0.018,-0.013*,-0.030***,-0.058***,-0.073***
,(0.010),(0.018),(0.013),(0.012),(0.007),(0.009),(0.016),(0.024)
β₂-β₁,-0.000,-0.004**,-0.003***,-0.001,0.001,-0.000,-0.004***,-0.003*
,(0.001),(0.002),(0.001),(0.001),(0.001),(0.001),(0.001),(0.002)
α₁,0.172***,0.095,0.322***,0.128**,0.299***,0.420***,0.318***,0.063
,(0.063),(0.062),(0.059),(0.064),(0.061),(0.057),(0.061),(0.064)
R-squared,0.039,0.035,0.164,0.041,0.101,0.217,0.152,0.036
R-squared Adj.,0.027,0.023,0.154,0.029,0.090,0.207,0.142,0.024


In [63]:
print(view.as_text())

                                             Main Model Results
               Armenia  Belarus  Kazakhstan Kyrgyz Republic Moldova  Russian Federation Tajikistan  Ukraine 
------------------------------------------------------------------------------------------------------------
const          0.070    0.205**  0.222***   0.088           0.067*   0.158***           0.262***   0.349*** 
               (0.048)  (0.080)  (0.063)    (0.060)         (0.035)  (0.049)            (0.072)    (0.117)  
β₁             -0.014   -0.047** -0.046***  -0.018          -0.013*  -0.030***          -0.058***  -0.073***
               (0.010)  (0.018)  (0.013)    (0.012)         (0.007)  (0.009)            (0.016)    (0.024)  
β₂-β₁          -0.000   -0.004** -0.003***  -0.001          0.001    -0.000             -0.004***  -0.003*  
               (0.001)  (0.002)  (0.001)    (0.001)         (0.001)  (0.001)            (0.001)    (0.002)  
α₁             0.172*** 0.095    0.322***   0.128**         0.29